In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AutoML Training with Pipeline - Tabular data regression
This notebook is simplified version of the below notebook in the official Google github. You can find more divese codes and detailed information from the link.
*  https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_automl_tabular.ipynb
*  https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/


### Install Python package


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

### Check package version

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.10.1
google_cloud_pipeline_components version: 2.18.0


### Set configuration

#### Authentication to access to the GCP

In [ ]:
import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}


Updated property [core/project].


### Import libraries

In [ ]:
from typing import NamedTuple

import google.cloud.aiplatform as aiplatform
import kfp
from google.cloud import bigquery
from kfp import compiler, dsl
from kfp.dsl import ( Artifact,
                      ClassificationMetrics,
                      Input,
                      Metrics,
                      Output,
                      component)

### Data Preparation

#### Create a bucket

In [ ]:
# Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-1209"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-1209/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-1209' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


#### Copy dta set into the bucket
*  gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/california_housing/california_housing_train.csv

In [ ]:
# ! gsutil cp gs://cloud-ml-data/NL-classification/happiness.csv {BUCKET_URI}/data/
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline/automl/"

TRAIN_FILE_NAME = "california_housing_train.csv"
! gsutil cp gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/california_housing/california_housing_train.csv {PIPELINE_ROOT}/data/

GCS_CSV_PATH = f"{PIPELINE_ROOT}/data/{TRAIN_FILE_NAME}"

Copying gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/california_housing/california_housing_train.csv [Content-Type=text/csv]...
/ [1 files][  1.6 MiB/  1.6 MiB]                                                
Operation completed over 1 objects/1.6 MiB.                                      


#### Allow allow to access to GCS

In [ ]:
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Configure pipeline

#### Initialize Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Define AutoML pipeline

In [ ]:

@kfp.dsl.pipeline(name="automl_tabular_regression_pipeline")
def automl_pipeline(project: str = PROJECT_ID, region: str = LOCATION):

    from google_cloud_pipeline_components.v1.automl.training_job import AutoMLTabularTrainingJobRunOp
    from google_cloud_pipeline_components.v1.dataset import TabularDatasetCreateOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)

    # https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.16.0/api/v1/dataset.html#v1.dataset.TabularDatasetCreateOp

    dataset_create_op = TabularDatasetCreateOp(
        project=project, display_name="automl_tabular_regression_pipeline", gcs_source=GCS_CSV_PATH
    )

    # https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.16.0/api/v1/automl/training_job.html#v1.automl.training_job.AutoMLTabularTrainingJobRunOp

    training_op = AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name="automl_tabular_regression_pipeline",
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse",
        column_transformations=[
            {"numeric": {"column_name": "longitude"}},
            {"numeric": {"column_name": "latitude"}},
            {"numeric": {"column_name": "housing_median_age"}},
            {"numeric": {"column_name": "total_rooms"}},
            {"numeric": {"column_name": "total_bedrooms"}},
            {"numeric": {"column_name": "population"}},
            {"numeric": {"column_name": "households"}},
            {"numeric": {"column_name": "median_income"}},
            {"numeric": {"column_name": "median_house_value"}},
        ],
        dataset=dataset_create_op.outputs["dataset"],
        target_column="median_house_value",
    )

    # https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.16.0/api/v1/endpoint.html#v1.endpoint.EndpointCreateOp

    endpoint_op = EndpointCreateOp(
        project=project,
        location=region,
        display_name="automl_tabular_regression_pipeline",
    )

    # https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.16.0/api/v1/endpoint.html#v1.endpoint.ModelDeployOp

    ModelDeployOp(
        model=training_op.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )



#### Compile the pipeline

In [ ]:

from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=automl_pipeline,
    package_path="automl_tabular_regression_pipeline.yaml",
)

### Run the pipeline

In [ ]:
import datetime

now = datetime.datetime.now()
now_format = now.strftime('%Y%m%d-%H%M%S')

DISPLAY_NAME = "automl_tabular_regression_pipeline" + now_format

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="automl_tabular_regression_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/721521243942/locations/us-central1/pipelineJobs/automl-tabular-regression-pipeline-20241208070003
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/721521243942/locations/us-central1/pipelineJobs/automl-tabular-regression-pipeline-20241208070003')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/automl-tabular-regression-pipeline-20241208070003?project=721521243942
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/721521243942/locations/us-central1/pipelineJobs/automl-tabular-regression-pipeline-20241208070003 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_job

### Cleaning up

In [ ]:
endpoint.undeploy(deployed_model_id=prediction.deployed_model_id)

In [ ]:
# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI